In [12]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# File paths for the two datasets
k2_file = r"C:\Users\mobed\Desktop\Nasa Space Apps\ML\raw\k2pandc_2025.10.01_03.31.10.csv"
kepler_file = r"C:\Users\mobed\Desktop\Nasa Space Apps\ML\raw\cumulative_2025.10.02_13.38.11.csv"

# Load the datasets
print("Loading K2 planets and candidates dataset...")
k2_df = pd.read_csv(k2_file)
print(f"K2 dataset shape: {k2_df.shape}")

print("\nLoading Kepler cumulative dataset...")
kepler_df = pd.read_csv(kepler_file)
print(f"Kepler dataset shape: {kepler_df.shape}")

print("\n" + "="*50)
print("DATASET OVERVIEW")
print("="*50)

Loading K2 planets and candidates dataset...
K2 dataset shape: (4004, 94)

Loading Kepler cumulative dataset...
Kepler dataset shape: (9564, 141)

DATASET OVERVIEW


In [13]:
# Examine columns in both datasets
print("K2 DATASET COLUMNS:")
print("-" * 30)
k2_columns = list(k2_df.columns)
for i, col in enumerate(k2_columns):
    print(f"{i+1:2d}. {col}")

print(f"\nTotal K2 columns: {len(k2_columns)}")

print("\n" + "="*50)
print("KEPLER DATASET COLUMNS:")
print("-" * 30)
kepler_columns = list(kepler_df.columns)
for i, col in enumerate(kepler_columns):
    print(f"{i+1:2d}. {col}")

print(f"\nTotal Kepler columns: {len(kepler_columns)}")

print("\n" + "="*50)
print("COMMON COLUMNS:")
print("-" * 30)
common_columns = set(k2_columns).intersection(set(kepler_columns))
print(f"Found {len(common_columns)} common columns:")
for col in sorted(common_columns):
    print(f"  - {col}")

K2 DATASET COLUMNS:
------------------------------
 1. pl_name
 2. hostname
 3. default_flag
 4. disposition
 5. disp_refname
 6. sy_snum
 7. sy_pnum
 8. discoverymethod
 9. disc_year
10. disc_facility
11. soltype
12. pl_controv_flag
13. pl_refname
14. pl_orbper
15. pl_orbpererr1
16. pl_orbpererr2
17. pl_orbperlim
18. pl_orbsmax
19. pl_orbsmaxerr1
20. pl_orbsmaxerr2
21. pl_orbsmaxlim
22. pl_rade
23. pl_radeerr1
24. pl_radeerr2
25. pl_radelim
26. pl_radj
27. pl_radjerr1
28. pl_radjerr2
29. pl_radjlim
30. pl_bmasse
31. pl_bmasseerr1
32. pl_bmasseerr2
33. pl_bmasselim
34. pl_bmassj
35. pl_bmassjerr1
36. pl_bmassjerr2
37. pl_bmassjlim
38. pl_bmassprov
39. pl_orbeccen
40. pl_orbeccenerr1
41. pl_orbeccenerr2
42. pl_orbeccenlim
43. pl_insol
44. pl_insolerr1
45. pl_insolerr2
46. pl_insollim
47. pl_eqt
48. pl_eqterr1
49. pl_eqterr2
50. pl_eqtlim
51. ttv_flag
52. st_refname
53. st_spectype
54. st_teff
55. st_tefferr1
56. st_tefferr2
57. st_tefflim
58. st_rad
59. st_raderr1
60. st_raderr2
61. st_

In [14]:
# Get basic info about both datasets
print("K2 DATASET INFO:")
print("-" * 20)
print(f"Shape: {k2_df.shape}")
print(f"Columns: {k2_df.shape[1]}")
print("Sample columns:", k2_df.columns.tolist()[:10])

print("\nKEPLER DATASET INFO:")
print("-" * 20)
print(f"Shape: {kepler_df.shape}")
print(f"Columns: {kepler_df.shape[1]}")
print("Sample columns:", kepler_df.columns.tolist()[:10])

# Check for common identifier columns
potential_ids = ['pl_name', 'hostname', 'kepid', 'kepoi_name', 'k2_name', 'epic_hostname']
common_ids = []
for id_col in potential_ids:
    if id_col in k2_df.columns and id_col in kepler_df.columns:
        common_ids.append(id_col)
    elif id_col in k2_df.columns:
        print(f"'{id_col}' found in K2 only")
    elif id_col in kepler_df.columns:
        print(f"'{id_col}' found in Kepler only")

print(f"\nCommon identifier columns: {common_ids}")

# Quick look at the first few rows to understand data structure
print(f"\nK2 first few columns preview:")
print(k2_df.iloc[:3, :5])

print(f"\nKepler first few columns preview:")
print(kepler_df.iloc[:3, :5])

K2 DATASET INFO:
--------------------
Shape: (4004, 94)
Columns: 94
Sample columns: ['pl_name', 'hostname', 'default_flag', 'disposition', 'disp_refname', 'sy_snum', 'sy_pnum', 'discoverymethod', 'disc_year', 'disc_facility']

KEPLER DATASET INFO:
--------------------
Shape: (9564, 141)
Columns: 141
Sample columns: ['rowid', 'kepid', 'kepoi_name', 'kepler_name', 'koi_disposition', 'koi_vet_stat', 'koi_vet_date', 'koi_pdisposition', 'koi_score', 'koi_fpflag_nt']
'pl_name' found in K2 only
'hostname' found in K2 only
'kepid' found in Kepler only
'kepoi_name' found in Kepler only

Common identifier columns: []

K2 first few columns preview:
             pl_name        hostname  default_flag disposition  \
0  EPIC 201111557.01  EPIC 201111557             1   CANDIDATE   
1  EPIC 201111557.01  EPIC 201111557             0   CANDIDATE   
2  EPIC 201126503.01  EPIC 201126503             1   CANDIDATE   

             disp_refname  
0  Livingston et al. 2018  
1  Livingston et al. 2018  
2  Va

In [15]:
# Create comprehensive column mapping based on documentation
# Mapping similar columns from both datasets to unified names

column_mapping = {
    # Planet identification
    'planet_name': {'k2': 'pl_name', 'kepler': 'kepler_name'},
    'host_star_name': {'k2': 'hostname', 'kepler': None},
    'kepid': {'k2': None, 'kepler': 'kepid'},
    'koi_name': {'k2': None, 'kepler': 'kepoi_name'},
    'k2_name': {'k2': 'k2_name', 'kepler': None},
    
    # Disposition and status
    'disposition': {'k2': 'disposition', 'kepler': 'koi_disposition'},
    'disposition_reference': {'k2': 'disp_refname', 'kepler': None},
    'project_disposition': {'k2': None, 'kepler': 'koi_pdisposition'},
    'disposition_score': {'k2': None, 'kepler': 'koi_score'},
    
    # System properties
    'num_stars': {'k2': 'sy_snum', 'kepler': None},
    'num_planets': {'k2': 'sy_pnum', 'kepler': None},
    
    # Discovery information
    'discovery_method': {'k2': 'discoverymethod', 'kepler': None},
    'discovery_year': {'k2': 'disc_year', 'kepler': None},
    'discovery_facility': {'k2': 'disc_facility', 'kepler': None},
    
    # Orbital parameters
    'orbital_period': {'k2': 'pl_orbper', 'kepler': 'koi_period'},
    'orbital_period_err1': {'k2': 'pl_orbpererr1', 'kepler': 'koi_period_err1'},
    'orbital_period_err2': {'k2': 'pl_orbpererr2', 'kepler': 'koi_period_err2'},
    'semi_major_axis': {'k2': 'pl_orbsmax', 'kepler': 'koi_sma'},
    'eccentricity': {'k2': 'pl_orbeccen', 'kepler': 'koi_eccen'},
    'inclination': {'k2': 'pl_orbincl', 'kepler': 'koi_incl'},
    
    # Planet physical properties
    'planet_radius_earth': {'k2': 'pl_rade', 'kepler': 'koi_prad'},
    'planet_radius_earth_err1': {'k2': 'pl_radeerr1', 'kepler': 'koi_prad_err1'},
    'planet_radius_earth_err2': {'k2': 'pl_radeerr2', 'kepler': 'koi_prad_err2'},
    'planet_radius_jupiter': {'k2': 'pl_radj', 'kepler': None},
    'planet_mass_earth': {'k2': 'pl_masse', 'kepler': None},
    'planet_mass_jupiter': {'k2': 'pl_massj', 'kepler': None},
    'equilibrium_temperature': {'k2': 'pl_eqt', 'kepler': 'koi_teq'},
    'insolation_flux': {'k2': 'pl_insol', 'kepler': 'koi_insol'},
    
    # Transit properties
    'transit_epoch': {'k2': 'pl_tranmid', 'kepler': 'koi_time0bk'},
    'transit_duration': {'k2': 'pl_trandur', 'kepler': 'koi_duration'},
    'transit_depth': {'k2': 'pl_trandep', 'kepler': 'koi_depth'},
    'impact_parameter': {'k2': 'pl_imppar', 'kepler': 'koi_impact'},
    
    # Stellar properties
    'stellar_effective_temp': {'k2': 'st_teff', 'kepler': 'koi_steff'},
    'stellar_radius': {'k2': 'st_rad', 'kepler': 'koi_srad'},
    'stellar_mass': {'k2': 'st_mass', 'kepler': 'koi_smass'},
    'stellar_metallicity': {'k2': 'st_met', 'kepler': 'koi_smet'},
    'stellar_surface_gravity': {'k2': 'st_logg', 'kepler': 'koi_slogg'},
    'stellar_age': {'k2': 'st_age', 'kepler': 'koi_sage'},
    
    # Photometry
    'kepler_magnitude': {'k2': 'sy_kepmag', 'kepler': 'koi_kepmag'},
    'v_magnitude': {'k2': 'sy_vmag', 'kepler': None},
    'j_magnitude': {'k2': 'sy_jmag', 'kepler': 'koi_jmag'},
    'h_magnitude': {'k2': 'sy_hmag', 'kepler': 'koi_hmag'},
    'k_magnitude': {'k2': 'sy_kmag', 'kepler': 'koi_kmag'},
    'gaia_magnitude': {'k2': 'sy_gaiamag', 'kepler': None},
    
    # Position
    'ra': {'k2': 'ra', 'kepler': 'ra'},
    'dec': {'k2': 'dec', 'kepler': 'dec'},
    'distance': {'k2': 'sy_dist', 'kepler': None},
    
    # Additional Kepler-specific columns that might be useful
    'max_single_event_stat': {'k2': None, 'kepler': 'koi_max_sngle_ev'},
    'max_multiple_event_stat': {'k2': None, 'kepler': 'koi_max_mult_ev'},
    'signal_to_noise': {'k2': None, 'kepler': 'koi_model_snr'},
    'num_transits': {'k2': None, 'kepler': 'koi_num_transits'},
}

print(f"Created mapping for {len(column_mapping)} unified columns")
print("Sample mappings:")
for i, (unified_name, mapping) in enumerate(list(column_mapping.items())[:10]):
    print(f"  {unified_name}: K2='{mapping['k2']}', Kepler='{mapping['kepler']}'")
    
print(f"\nColumns available in K2 only: {sum(1 for m in column_mapping.values() if m['k2'] and not m['kepler'])}")
print(f"Columns available in Kepler only: {sum(1 for m in column_mapping.values() if m['kepler'] and not m['k2'])}")
print(f"Columns available in both: {sum(1 for m in column_mapping.values() if m['k2'] and m['kepler'])}")

Created mapping for 51 unified columns
Sample mappings:
  planet_name: K2='pl_name', Kepler='kepler_name'
  host_star_name: K2='hostname', Kepler='None'
  kepid: K2='None', Kepler='kepid'
  koi_name: K2='None', Kepler='kepoi_name'
  k2_name: K2='k2_name', Kepler='None'
  disposition: K2='disposition', Kepler='koi_disposition'
  disposition_reference: K2='disp_refname', Kepler='None'
  project_disposition: K2='None', Kepler='koi_pdisposition'
  disposition_score: K2='None', Kepler='koi_score'
  num_stars: K2='sy_snum', Kepler='None'

Columns available in K2 only: 14
Columns available in Kepler only: 8
Columns available in both: 29


In [16]:
def create_unified_dataframe(df, dataset_name, column_mapping):
    """
    Create a unified dataframe using the column mapping
    """
    unified_df = pd.DataFrame()
    
    # Add dataset source identifier
    unified_df['data_source'] = dataset_name
    
    # Map columns according to the mapping
    for unified_col, mapping in column_mapping.items():
        source_col = mapping[dataset_name.lower()]
        if source_col and source_col in df.columns:
            unified_df[unified_col] = df[source_col]
        else:
            unified_df[unified_col] = np.nan
    
    return unified_df

# Create unified dataframes for both datasets
print("Creating unified K2 dataframe...")
k2_unified = create_unified_dataframe(k2_df, 'K2', column_mapping)
print(f"K2 unified shape: {k2_unified.shape}")

print("\nCreating unified Kepler dataframe...")
kepler_unified = create_unified_dataframe(kepler_df, 'Kepler', column_mapping)
print(f"Kepler unified shape: {kepler_unified.shape}")

# Check which columns have data in each dataset
print("\nData availability summary:")
print("-" * 40)

k2_available = []
kepler_available = []
both_available = []
neither_available = []

for col in column_mapping.keys():
    k2_has_data = not k2_unified[col].isna().all()
    kepler_has_data = not kepler_unified[col].isna().all()
    
    if k2_has_data and kepler_has_data:
        both_available.append(col)
    elif k2_has_data:
        k2_available.append(col)
    elif kepler_has_data:
        kepler_available.append(col)
    else:
        neither_available.append(col)

print(f"Columns with data in both datasets: {len(both_available)}")
print(f"Columns with data only in K2: {len(k2_available)}")
print(f"Columns with data only in Kepler: {len(kepler_available)}")
print(f"Columns with no data: {len(neither_available)}")

# Show sample of available columns
print(f"\nSample columns available in both:")
for col in both_available[:10]:
    print(f"  - {col}")

print(f"\nSample K2-only columns:")
for col in k2_available[:5]:
    print(f"  - {col}")
    
print(f"\nSample Kepler-only columns:")
for col in kepler_available[:5]:
    print(f"  - {col}")

Creating unified K2 dataframe...
K2 unified shape: (4004, 52)

Creating unified Kepler dataframe...
Kepler unified shape: (9564, 52)

Data availability summary:
----------------------------------------
Columns with data in both datasets: 20
Columns with data only in K2: 11
Columns with data only in Kepler: 16
Columns with no data: 4

Sample columns available in both:
  - planet_name
  - disposition
  - orbital_period
  - orbital_period_err1
  - orbital_period_err2
  - semi_major_axis
  - eccentricity
  - planet_radius_earth
  - planet_radius_earth_err1
  - planet_radius_earth_err2

Sample K2-only columns:
  - host_star_name
  - disposition_reference
  - num_stars
  - num_planets
  - discovery_method

Sample Kepler-only columns:
  - kepid
  - koi_name
  - project_disposition
  - disposition_score
  - inclination


In [17]:
# Data cleaning and standardization
print("CLEANING AND STANDARDIZING DATA")
print("=" * 40)

# Check data types and clean up
def clean_dataframe(df, dataset_name):
    """Clean and standardize the dataframe"""
    df_clean = df.copy()
    
    # Convert numeric columns that might be stored as strings
    numeric_columns = [
        'orbital_period', 'orbital_period_err1', 'orbital_period_err2',
        'semi_major_axis', 'eccentricity', 'inclination',
        'planet_radius_earth', 'planet_radius_earth_err1', 'planet_radius_earth_err2',
        'equilibrium_temperature', 'insolation_flux',
        'transit_epoch', 'transit_duration', 'transit_depth', 'impact_parameter',
        'stellar_effective_temp', 'stellar_radius', 'stellar_mass',
        'stellar_metallicity', 'stellar_surface_gravity', 'stellar_age',
        'kepler_magnitude', 'v_magnitude', 'j_magnitude', 'h_magnitude', 'k_magnitude',
        'ra', 'dec', 'distance', 'max_single_event_stat', 'max_multiple_event_stat',
        'signal_to_noise', 'num_transits'
    ]
    
    for col in numeric_columns:
        if col in df_clean.columns:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    
    # Standardize disposition values
    if 'disposition' in df_clean.columns:
        disposition_mapping = {
            'CONFIRMED': 'CONFIRMED',
            'CANDIDATE': 'CANDIDATE', 
            'FALSE POSITIVE': 'FALSE POSITIVE',
            'REFUTED': 'FALSE POSITIVE',  # Standardize refuted to false positive
            'NOT DISPOSITIONED': 'NOT DISPOSITIONED'
        }
        df_clean['disposition'] = df_clean['disposition'].map(disposition_mapping).fillna(df_clean['disposition'])
    
    print(f"{dataset_name} cleaning completed:")
    print(f"  - Rows: {len(df_clean)}")
    print(f"  - Columns with data: {sum(~df_clean.isna().all())}")
    
    return df_clean

# Clean both datasets
print("Cleaning K2 data...")
k2_clean = clean_dataframe(k2_unified, "K2")

print("\nCleaning Kepler data...")
kepler_clean = clean_dataframe(kepler_unified, "Kepler")

# Check for any data quality issues
print(f"\nDATA QUALITY CHECK:")
print("-" * 25)

# Check for duplicates within each dataset
k2_dupes = k2_clean.duplicated().sum()
kepler_dupes = kepler_clean.duplicated().sum()
print(f"K2 duplicate rows: {k2_dupes}")
print(f"Kepler duplicate rows: {kepler_dupes}")

# Check disposition distribution
print(f"\nDisposition distribution:")
if 'disposition' in k2_clean.columns:
    print("K2:")
    print(k2_clean['disposition'].value_counts())
    
if 'disposition' in kepler_clean.columns:
    print("\nKepler:")
    print(kepler_clean['disposition'].value_counts())

CLEANING AND STANDARDIZING DATA
Cleaning K2 data...
K2 cleaning completed:
  - Rows: 4004
  - Columns with data: 31

Cleaning Kepler data...
Kepler cleaning completed:
  - Rows: 9564
  - Columns with data: 36

DATA QUALITY CHECK:
-------------------------
K2 duplicate rows: 28
Kepler duplicate rows: 0

Disposition distribution:
K2:
disposition
CONFIRMED         2315
CANDIDATE         1374
FALSE POSITIVE     315
Name: count, dtype: int64

Kepler:
disposition
FALSE POSITIVE    4839
CONFIRMED         2746
CANDIDATE         1979
Name: count, dtype: int64


In [18]:
# Merge the datasets
print("MERGING DATASETS")
print("=" * 20)

# Combine both cleaned datasets
master_df = pd.concat([k2_clean, kepler_clean], ignore_index=True, sort=False)

print(f"Master dataset created!")
print(f"Total rows: {len(master_df)}")
print(f"Total columns: {len(master_df.columns)}")

# Add some useful computed columns
print(f"\nAdding computed columns...")

# Create a unique identifier for tracking
master_df['row_id'] = range(1, len(master_df) + 1)

# Add mission information based on data source
master_df['mission'] = master_df['data_source'].map({
    'K2': 'K2',
    'Kepler': 'Kepler'
})

# Create a combined identifier when possible
def create_combined_id(row):
    if pd.notna(row['planet_name']):
        return row['planet_name']
    elif pd.notna(row['koi_name']):
        return row['koi_name']
    elif pd.notna(row['k2_name']):
        return row['k2_name']
    else:
        return f"{row['data_source']}_{row['row_id']}"

master_df['combined_id'] = master_df.apply(create_combined_id, axis=1)

# Reorder columns for better organization
priority_columns = [
    'row_id', 'combined_id', 'data_source', 'mission',
    'planet_name', 'host_star_name', 'kepid', 'koi_name', 'k2_name',
    'disposition', 'project_disposition', 'disposition_score',
    'orbital_period', 'semi_major_axis', 'eccentricity', 'inclination',
    'planet_radius_earth', 'equilibrium_temperature', 'insolation_flux',
    'transit_duration', 'transit_depth', 'impact_parameter',
    'stellar_effective_temp', 'stellar_radius', 'stellar_mass',
    'kepler_magnitude', 'ra', 'dec'
]

# Get remaining columns
remaining_columns = [col for col in master_df.columns if col not in priority_columns]

# Reorder
final_column_order = priority_columns + remaining_columns
master_df = master_df[final_column_order]

print(f"Dataset organization completed!")
print(f"Final shape: {master_df.shape}")

# Save information about data sources
data_source_summary = master_df['data_source'].value_counts()
print(f"\nData source distribution:")
for source, count in data_source_summary.items():
    print(f"  {source}: {count:,} objects")

# Check disposition distribution in merged dataset
print(f"\nCombined disposition distribution:")
disposition_summary = master_df['disposition'].value_counts()
for disp, count in disposition_summary.items():
    print(f"  {disp}: {count:,} objects")

MERGING DATASETS
Master dataset created!
Total rows: 13568
Total columns: 52

Adding computed columns...
Dataset organization completed!
Final shape: (13568, 55)

Data source distribution:

Combined disposition distribution:
  FALSE POSITIVE: 5,154 objects
  CONFIRMED: 5,061 objects
  CANDIDATE: 3,353 objects


In [19]:
# Final validation and summary
print("MASTER DATASET VALIDATION & SUMMARY")
print("=" * 40)

# Basic statistics
print(f"📊 DATASET OVERVIEW:")
print(f"   Total objects: {len(master_df):,}")
print(f"   Total features: {len(master_df.columns)}")
print(f"   K2 objects: {len(master_df[master_df['data_source'] == 'K2']):,}")
print(f"   Kepler objects: {len(master_df[master_df['data_source'] == 'Kepler']):,}")

# Check data completeness for key columns
print(f"\n📈 DATA COMPLETENESS (% of non-null values):")
key_columns = [
    'planet_name', 'disposition', 'orbital_period', 'planet_radius_earth',
    'stellar_effective_temp', 'kepler_magnitude', 'ra', 'dec'
]

for col in key_columns:
    if col in master_df.columns:
        completeness = (1 - master_df[col].isna().sum() / len(master_df)) * 100
        print(f"   {col}: {completeness:.1f}%")

# Show sample data
print(f"\n📋 SAMPLE DATA (first 5 rows):")
sample_cols = ['combined_id', 'data_source', 'disposition', 'orbital_period', 
               'planet_radius_earth', 'stellar_effective_temp']
available_sample_cols = [col for col in sample_cols if col in master_df.columns]
print(master_df[available_sample_cols].head())

# Summary statistics for numerical columns
print(f"\n📊 NUMERICAL SUMMARY:")
numerical_cols = ['orbital_period', 'planet_radius_earth', 'equilibrium_temperature', 
                  'stellar_effective_temp', 'stellar_radius', 'stellar_mass']
available_numerical_cols = [col for col in numerical_cols if col in master_df.columns]

if available_numerical_cols:
    summary_stats = master_df[available_numerical_cols].describe()
    print(summary_stats.round(3))

# Check for potential overlaps between datasets
print(f"\n🔍 OVERLAP ANALYSIS:")
k2_planets = set(master_df[master_df['data_source'] == 'K2']['planet_name'].dropna())
kepler_planets = set(master_df[master_df['data_source'] == 'Kepler']['planet_name'].dropna())
overlapping_names = k2_planets.intersection(kepler_planets)
print(f"   Potential overlapping planet names: {len(overlapping_names)}")
if overlapping_names:
    print(f"   Examples: {list(overlapping_names)[:5]}")

# Column availability summary
print(f"\n📋 COLUMN AVAILABILITY BY SOURCE:")
for source in ['K2', 'Kepler']:
    source_data = master_df[master_df['data_source'] == source]
    available_cols = sum(~source_data.isna().all())
    print(f"   {source}: {available_cols} columns with data")

print(f"\n✅ MASTER DATASET SUCCESSFULLY CREATED!")
print(f"   Ready for analysis with {len(master_df):,} exoplanet objects")
print(f"   Contains data from both K2 and Kepler missions")
print(f"   Unified schema with {len(master_df.columns)} total features")

MASTER DATASET VALIDATION & SUMMARY
📊 DATASET OVERVIEW:
   Total objects: 13,568
   Total features: 55
   K2 objects: 0
   Kepler objects: 0

📈 DATA COMPLETENESS (% of non-null values):
   planet_name: 49.8%
   disposition: 100.0%
   orbital_period: 99.7%
   planet_radius_earth: 91.2%
   stellar_effective_temp: 89.1%
   kepler_magnitude: 70.5%
   ra: 100.0%
   dec: 100.0%

📋 SAMPLE DATA (first 5 rows):
         combined_id data_source disposition  orbital_period  \
0  EPIC 201111557.01         NaN   CANDIDATE        2.301830   
1  EPIC 201111557.01         NaN   CANDIDATE        2.302368   
2  EPIC 201126503.01         NaN   CANDIDATE        1.194749   
3  EPIC 201127519.01         NaN   CANDIDATE        6.178369   
4  EPIC 201127519.01         NaN   CANDIDATE        6.178870   

   planet_radius_earth  stellar_effective_temp  
0             1.120000                 4616.52  
1             1.312588                 4720.00  
2             4.190000                 3919.00  
3            

In [20]:
# Display final dataset info and save option
print("🎯 FINAL MASTER DATASET INFORMATION")
print("=" * 45)

print(f"Dataset Name: Merged Exoplanet Master Dataset")
print(f"Sources: K2 Planets & Candidates + Kepler Cumulative")
print(f"Total Records: {len(master_df):,}")
print(f"Total Features: {len(master_df.columns)}")

print(f"\n📊 Record Distribution:")
print(f"   K2 Mission: {len(master_df[master_df['data_source'] == 'K2']):,} objects")
print(f"   Kepler Mission: {len(master_df[master_df['data_source'] == 'Kepler']):,} objects")

print(f"\n🏷️ All Available Columns:")
for i, col in enumerate(master_df.columns, 1):
    print(f"{i:2d}. {col}")

# Show a few example records to verify structure
print(f"\n🔍 Sample Records:")
print("="*80)
sample_df = master_df[['combined_id', 'data_source', 'disposition', 'orbital_period', 
                       'planet_radius_earth', 'stellar_effective_temp']].head(3)
print(sample_df.to_string(index=False))

print(f"\n💾 To save this master dataset:")
print(f"   master_df.to_csv('exoplanet_master_dataset.csv', index=False)")
print(f"   # This will create a unified CSV with all {len(master_df):,} objects")

# Quick data type summary
print(f"\n📋 Data Types Summary:")
dtypes_summary = master_df.dtypes.value_counts()
for dtype, count in dtypes_summary.items():
    print(f"   {dtype}: {count} columns")

print(f"\n✨ SUCCESS: Master dataset ready for machine learning and analysis!")

🎯 FINAL MASTER DATASET INFORMATION
Dataset Name: Merged Exoplanet Master Dataset
Sources: K2 Planets & Candidates + Kepler Cumulative
Total Records: 13,568
Total Features: 55

📊 Record Distribution:
   K2 Mission: 0 objects
   Kepler Mission: 0 objects

🏷️ All Available Columns:
 1. row_id
 2. combined_id
 3. data_source
 4. mission
 5. planet_name
 6. host_star_name
 7. kepid
 8. koi_name
 9. k2_name
10. disposition
11. project_disposition
12. disposition_score
13. orbital_period
14. semi_major_axis
15. eccentricity
16. inclination
17. planet_radius_earth
18. equilibrium_temperature
19. insolation_flux
20. transit_duration
21. transit_depth
22. impact_parameter
23. stellar_effective_temp
24. stellar_radius
25. stellar_mass
26. kepler_magnitude
27. ra
28. dec
29. disposition_reference
30. num_stars
31. num_planets
32. discovery_method
33. discovery_year
34. discovery_facility
35. orbital_period_err1
36. orbital_period_err2
37. planet_radius_earth_err1
38. planet_radius_earth_err2
39. p

In [21]:
master_df.describe()

,row_id,kepid,k2_name,disposition_score,orbital_period,semi_major_axis,eccentricity,inclination,planet_radius_earth,equilibrium_temperature,...,v_magnitude,j_magnitude,h_magnitude,k_magnitude,gaia_magnitude,distance,max_single_event_stat,max_multiple_event_stat,signal_to_noise,num_transits
count,13568.000000,9.564000e+03,0.0,8054.000000,13524.000000,10021.000000,9630.000000,9200.000000,12374.000000,10055.000000,...,3962.000000,9539.000000,9539.000000,13520.000000,3948.000000,3879.000000,8422.000000,8422.000000,9201.000000,8422.000000
mean,6784.500000,7.690628e+06,NaN,0.480829,65.302494,0.214238,0.005379,82.469147,78.677536,1070.309560,...,13.174241,12.993311,12.620604,12.003345,12.837391,391.104905,176.846052,1025.664672,259.895001,385.006768
std,3916.888561,2.653459e+06,NaN,0.476928,1337.141748,0.549591,0.039040,15.223627,2654.199983,830.871608,...,1.900071,1.291912,1.267215,1.555375,1.737748,543.808755,770.902357,4154.121620,795.806615,545.756200
min,1.000000,7.574500e+05,NaN,0.000000,0.175660,0.003400,0.000000,2.290000,0.080000,25.000000,...,5.840000,4.097000,3.014000,2.311000,5.808980,21.818200,2.417437,7.105086,0.000000,0.000000
25%,3392.750000,5.556034e+06,NaN,0.000000,2.790824,0.038170,0.000000,83.920000,1.500000,546.000000,...,12.059750,12.253000,11.914500,10.995000,11.803625,156.110500,3.997856,10.733030,12.000000,41.000000
50%,6784.500000,7.906892e+06,NaN,0.334000,8.328175,0.081800,0.000000,88.500000,2.470000,871.000000,...,12.975500,13.236000,12.834000,12.226000,12.670200,264.780000,5.589751,19.254411,23.000000,143.000000
75%,10176.250000,9.873066e+06,NaN,0.998000,24.948485,0.195800,0.000000,89.770000,10.495000,1352.000000,...,14.314000,13.968000,13.551000,13.238000,13.917550,446.821000,16.947631,71.998003,78.000000,469.000000
max,13568.000000,1.293514e+07,NaN,1.000000,129995.778400,44.989200,0.853000,90.000000,200346.000000,14667.000000,...,20.556100,17.372000,17.615000,17.038000,20.247800,9319.510000,22982.162000,120049.680000,9054.700000,2664.000000


In [22]:
# Check disposition category in the master dataset
print("DISPOSITION CATEGORY ANALYSIS")
print("=" * 35)

# Check if disposition column exists and show its values
if 'disposition' in master_df.columns:
    print("✅ Disposition column found!")
    print(f"\nDisposition value counts:")
    disp_counts = master_df['disposition'].value_counts(dropna=False)
    print(disp_counts)
    
    print(f"\nDisposition distribution by data source:")
    disposition_by_source = pd.crosstab(master_df['data_source'], master_df['disposition'], margins=True)
    print(disposition_by_source)
    
    print(f"\nSample records with disposition:")
    sample_disp = master_df[['combined_id', 'data_source', 'disposition', 'planet_name', 'koi_name']].head(10)
    print(sample_disp)
    
    # Check for any null values
    null_count = master_df['disposition'].isna().sum()
    print(f"\nNull disposition values: {null_count}")
    
else:
    print("❌ Disposition column not found!")
    print("Available columns containing 'disp':")
    disp_columns = [col for col in master_df.columns if 'disp' in col.lower()]
    for col in disp_columns:
        print(f"  - {col}")

DISPOSITION CATEGORY ANALYSIS
✅ Disposition column found!

Disposition value counts:
disposition
FALSE POSITIVE    5154
CONFIRMED         5061
CANDIDATE         3353
Name: count, dtype: int64

Disposition distribution by data source:
Empty DataFrame
Columns: []
Index: []

Sample records with disposition:
         combined_id data_source disposition        planet_name koi_name
0  EPIC 201111557.01         NaN   CANDIDATE  EPIC 201111557.01      NaN
1  EPIC 201111557.01         NaN   CANDIDATE  EPIC 201111557.01      NaN
2  EPIC 201126503.01         NaN   CANDIDATE  EPIC 201126503.01      NaN
3  EPIC 201127519.01         NaN   CANDIDATE  EPIC 201127519.01      NaN
4  EPIC 201127519.01         NaN   CANDIDATE  EPIC 201127519.01      NaN
5  EPIC 201147085.01         NaN   CANDIDATE  EPIC 201147085.01      NaN
6  EPIC 201152065.01         NaN   CANDIDATE  EPIC 201152065.01      NaN
7  EPIC 201160662.01         NaN   CANDIDATE  EPIC 201160662.01      NaN
8  EPIC 201164625.01         NaN   CA

In [23]:
# Fix the data_source issue and show proper disposition breakdown
print("FIXING DATA SOURCE AND ANALYZING DISPOSITION")
print("=" * 45)

# Check the current data_source column
print("Current data_source values:")
print(master_df['data_source'].value_counts(dropna=False))

# Let's check if we can identify K2 vs Kepler records by looking at specific columns
print(f"\nChecking for K2-specific vs Kepler-specific identifiers:")

# K2 records should have k2_name or EPIC in planet_name
k2_mask = (master_df['k2_name'].notna()) | (master_df['planet_name'].str.contains('EPIC', na=False))
kepler_mask = (master_df['kepid'].notna()) | (master_df['koi_name'].notna())

print(f"Records with K2 identifiers: {k2_mask.sum()}")
print(f"Records with Kepler identifiers: {kepler_mask.sum()}")
print(f"Records with both: {(k2_mask & kepler_mask).sum()}")
print(f"Records with neither: {(~k2_mask & ~kepler_mask).sum()}")

# Fix the data_source assignment
master_df_fixed = master_df.copy()

# Assign data sources based on identifiers
master_df_fixed.loc[k2_mask & ~kepler_mask, 'data_source'] = 'K2'
master_df_fixed.loc[kepler_mask & ~k2_mask, 'data_source'] = 'Kepler'
master_df_fixed.loc[k2_mask & kepler_mask, 'data_source'] = 'Both'  # If somehow both

print(f"\nFixed data_source values:")
print(master_df_fixed['data_source'].value_counts(dropna=False))

# Now show disposition by corrected data source
print(f"\nDisposition distribution by corrected data source:")
if master_df_fixed['data_source'].notna().any():
    disposition_by_source_fixed = pd.crosstab(
        master_df_fixed['data_source'], 
        master_df_fixed['disposition'], 
        margins=True
    )
    print(disposition_by_source_fixed)

# Update the master dataframe
master_df = master_df_fixed.copy()
print(f"\n✅ Master dataframe updated with corrected data sources!")

FIXING DATA SOURCE AND ANALYZING DISPOSITION
Current data_source values:
data_source
NaN    13568
Name: count, dtype: int64

Checking for K2-specific vs Kepler-specific identifiers:
Records with K2 identifiers: 1791
Records with Kepler identifiers: 9564
Records with both: 0
Records with neither: 2213

Fixed data_source values:
data_source
Kepler    9564
NaN       2213
K2        1791
Name: count, dtype: int64

Disposition distribution by corrected data source:
disposition  CANDIDATE  CONFIRMED  FALSE POSITIVE    All
data_source                                             
K2                1373        125             293   1791
Kepler            1979       2746            4839   9564
All               3352       2871            5132  11355

✅ Master dataframe updated with corrected data sources!


In [24]:
# COMPREHENSIVE DISPOSITION ANALYSIS - TRY AGAIN
print("🎯 COMPREHENSIVE DISPOSITION CATEGORY ANALYSIS")
print("=" * 50)

# Check current state of master_df
print(f"Current master_df shape: {master_df.shape}")
print(f"Data source distribution:")
print(master_df['data_source'].value_counts(dropna=False))

# First, let's properly fix the data source issue by going back to the original approach
print(f"\n🔧 RECONSTRUCTING PROPER DATA SOURCES...")

# Check what we have in the original dataframes
print(f"Original K2 shape: {k2_df.shape}")
print(f"Original Kepler shape: {kepler_df.shape}")

# Let's rebuild the master dataframe with proper source tracking
print(f"\n🏗️ REBUILDING MASTER DATAFRAME WITH PROPER TRACKING...")

# Create K2 subset with source marking
k2_master = k2_clean.copy()
k2_master['data_source'] = 'K2'
k2_master['mission'] = 'K2'

# Create Kepler subset with source marking  
kepler_master = kepler_clean.copy()
kepler_master['data_source'] = 'Kepler'
kepler_master['mission'] = 'Kepler'

# Combine properly
master_df_rebuilt = pd.concat([k2_master, kepler_master], ignore_index=True, sort=False)

# Add row IDs and combined IDs
master_df_rebuilt['row_id'] = range(1, len(master_df_rebuilt) + 1)

def create_combined_id_fixed(row):
    if pd.notna(row['planet_name']):
        return row['planet_name']
    elif pd.notna(row['koi_name']):
        return row['koi_name']
    elif pd.notna(row['k2_name']):
        return row['k2_name']
    else:
        return f"{row['data_source']}_{row['row_id']}"

master_df_rebuilt['combined_id'] = master_df_rebuilt.apply(create_combined_id_fixed, axis=1)

# Update the global master_df
master_df = master_df_rebuilt.copy()

print(f"✅ Rebuilt master dataframe!")
print(f"New shape: {master_df.shape}")
print(f"Data source distribution:")
print(master_df['data_source'].value_counts())

# NOW ANALYZE DISPOSITION PROPERLY
print(f"\n📊 DISPOSITION ANALYSIS:")
print("-" * 30)

print(f"Overall disposition counts:")
overall_disp = master_df['disposition'].value_counts(dropna=False)
print(overall_disp)

print(f"\nDisposition by data source:")
disp_crosstab = pd.crosstab(master_df['data_source'], master_df['disposition'], margins=True)
print(disp_crosstab)

print(f"\nDisposition percentages by source:")
disp_pct = pd.crosstab(master_df['data_source'], master_df['disposition'], normalize='index') * 100
print(disp_pct.round(1))

# Show examples from each source
print(f"\n📋 SAMPLE RECORDS BY SOURCE:")
print("K2 Examples:")
k2_sample = master_df[master_df['data_source'] == 'K2'][['combined_id', 'disposition', 'planet_name']].head()
print(k2_sample)

print(f"\nKepler Examples:")
kepler_sample = master_df[master_df['data_source'] == 'Kepler'][['combined_id', 'disposition', 'koi_name']].head()
print(kepler_sample)

🎯 COMPREHENSIVE DISPOSITION CATEGORY ANALYSIS
Current master_df shape: (13568, 55)
Data source distribution:
data_source
Kepler    9564
NaN       2213
K2        1791
Name: count, dtype: int64

🔧 RECONSTRUCTING PROPER DATA SOURCES...
Original K2 shape: (4004, 94)
Original Kepler shape: (9564, 141)

🏗️ REBUILDING MASTER DATAFRAME WITH PROPER TRACKING...
✅ Rebuilt master dataframe!
New shape: (13568, 55)
Data source distribution:
data_source
Kepler    9564
K2        4004
Name: count, dtype: int64

📊 DISPOSITION ANALYSIS:
------------------------------
Overall disposition counts:
disposition
FALSE POSITIVE    5154
CONFIRMED         5061
CANDIDATE         3353
Name: count, dtype: int64

Disposition by data source:
disposition  CANDIDATE  CONFIRMED  FALSE POSITIVE    All
data_source                                             
K2                1374       2315             315   4004
Kepler            1979       2746            4839   9564
All               3353       5061            5154  13

In [25]:
# Filter dataset to keep only CANDIDATE and CONFIRMED objects
print("🔍 FILTERING DATASET - KEEPING ONLY CANDIDATES AND CONFIRMED PLANETS")
print("=" * 65)

# Show current distribution
print("Current disposition distribution:")
current_disp = master_df['disposition'].value_counts()
print(current_disp)
print(f"Total objects: {len(master_df):,}")

# Filter to keep only CANDIDATE and CONFIRMED
filtered_dispositions = ['CANDIDATE', 'CONFIRMED']
master_df_filtered = master_df[master_df['disposition'].isin(filtered_dispositions)].copy()

print(f"\n✂️ FILTERING RESULTS:")
print("-" * 25)
print(f"Objects before filtering: {len(master_df):,}")
print(f"Objects after filtering: {len(master_df_filtered):,}")
print(f"Objects removed (FALSE POSITIVE): {len(master_df) - len(master_df_filtered):,}")
print(f"Retention rate: {(len(master_df_filtered) / len(master_df) * 100):.1f}%")

# Show new distribution
print(f"\n📊 NEW DISPOSITION DISTRIBUTION:")
print("-" * 35)
new_disp = master_df_filtered['disposition'].value_counts()
print(new_disp)

# Show breakdown by data source
print(f"\nDistribution by data source:")
filtered_crosstab = pd.crosstab(master_df_filtered['data_source'], master_df_filtered['disposition'], margins=True)
print(filtered_crosstab)

print(f"\nPercentages by source:")
filtered_pct = pd.crosstab(master_df_filtered['data_source'], master_df_filtered['disposition'], normalize='index') * 100
print(filtered_pct.round(1))

# Update the master dataframe
master_df = master_df_filtered.copy()

# Reset row IDs after filtering
master_df['row_id'] = range(1, len(master_df) + 1)

print(f"\n✅ DATASET SUCCESSFULLY FILTERED!")
print(f"Final dataset contains {len(master_df):,} potential planets (candidates + confirmed)")
print(f"Shape: {master_df.shape}")

# Show sample of filtered data
print(f"\n📋 SAMPLE OF FILTERED DATA:")
sample_filtered = master_df[['row_id', 'combined_id', 'data_source', 'disposition', 'planet_radius_earth', 'orbital_period']].head(10)
print(sample_filtered)

🔍 FILTERING DATASET - KEEPING ONLY CANDIDATES AND CONFIRMED PLANETS
Current disposition distribution:
disposition
FALSE POSITIVE    5154
CONFIRMED         5061
CANDIDATE         3353
Name: count, dtype: int64
Total objects: 13,568

✂️ FILTERING RESULTS:
-------------------------
Objects before filtering: 13,568
Objects after filtering: 8,414
Objects removed (FALSE POSITIVE): 5,154
Retention rate: 62.0%

📊 NEW DISPOSITION DISTRIBUTION:
-----------------------------------
disposition
CONFIRMED    5061
CANDIDATE    3353
Name: count, dtype: int64

Distribution by data source:
disposition  CANDIDATE  CONFIRMED   All
data_source                            
K2                1374       2315  3689
Kepler            1979       2746  4725
All               3353       5061  8414

Percentages by source:
disposition  CANDIDATE  CONFIRMED
data_source                      
K2                37.2       62.8
Kepler            41.9       58.1

✅ DATASET SUCCESSFULLY FILTERED!
Final dataset contains 8,41

In [26]:
# TYPE CONVERSION AND COLUMN CLEANING
print("🔄 TYPE CONVERSION AND COLUMN CLEANING")
print("=" * 45)

# First, let's analyze the current state
print(f"Starting dataset shape: {master_df.shape}")
print(f"Starting columns: {len(master_df.columns)}")

# Check data types
print(f"\nCurrent data types:")
current_dtypes = master_df.dtypes.value_counts()
print(current_dtypes)

# Analyze missing values per column
print(f"\n📊 MISSING VALUES ANALYSIS:")
print("-" * 35)
missing_analysis = pd.DataFrame({
    'Column': master_df.columns,
    'Missing_Count': master_df.isnull().sum(),
    'Missing_Percentage': (master_df.isnull().sum() / len(master_df)) * 100,
    'Data_Type': master_df.dtypes,
    'Non_Null_Count': master_df.count()
})
missing_analysis = missing_analysis.sort_values('Missing_Percentage', ascending=False)

# Show columns with high missing percentages
high_missing = missing_analysis[missing_analysis['Missing_Percentage'] > 90]
print(f"Columns with >90% missing values ({len(high_missing)} columns):")
for _, row in high_missing.iterrows():
    print(f"  {row['Column']}: {row['Missing_Percentage']:.1f}% missing")

medium_missing = missing_analysis[(missing_analysis['Missing_Percentage'] > 50) & (missing_analysis['Missing_Percentage'] <= 90)]
print(f"\nColumns with 50-90% missing values ({len(medium_missing)} columns):")
for _, row in medium_missing.head(10).iterrows():
    print(f"  {row['Column']}: {row['Missing_Percentage']:.1f}% missing")

# Check for zero-only columns (excluding ID columns)
print(f"\n🔍 CHECKING FOR ZERO-ONLY COLUMNS:")
print("-" * 35)
numeric_cols = master_df.select_dtypes(include=[np.number]).columns
zero_only_cols = []

for col in numeric_cols:
    if col not in ['row_id']:  # Exclude ID columns
        non_null_values = master_df[col].dropna()
        if len(non_null_values) > 0 and (non_null_values == 0).all():
            zero_only_cols.append(col)

print(f"Columns with only zeros: {len(zero_only_cols)}")
for col in zero_only_cols:
    print(f"  - {col}")

# Check for columns with zeros + NAs only
print(f"\n🔍 CHECKING FOR ZERO+NA ONLY COLUMNS:")
print("-" * 40)
zero_na_only_cols = []

for col in numeric_cols:
    if col not in ['row_id']:
        non_null_values = master_df[col].dropna()
        if len(non_null_values) > 0:
            unique_values = set(non_null_values.unique())
            if unique_values == {0} or unique_values == {0.0} or len(unique_values) == 0:
                zero_na_only_cols.append(col)

print(f"Columns with only zeros and NAs: {len(zero_na_only_cols)}")
for col in zero_na_only_cols:
    non_null_count = master_df[col].count()
    print(f"  - {col}: {non_null_count} non-null values, all zeros")

🔄 TYPE CONVERSION AND COLUMN CLEANING
Starting dataset shape: (8414, 55)
Starting columns: 55

Current data types:
float64    43
object     11
int64       1
Name: count, dtype: int64

📊 MISSING VALUES ANALYSIS:
-----------------------------------
Columns with >90% missing values (4 columns):
  k2_name: 100.0% missing
  planet_mass_earth: 100.0% missing
  planet_mass_jupiter: 100.0% missing
  stellar_age: 100.0% missing

Columns with 50-90% missing values (12 columns):
  planet_radius_jupiter: 63.5% missing
  distance: 57.4% missing
  gaia_magnitude: 56.7% missing
  v_magnitude: 56.4% missing
  discovery_facility: 56.2% missing
  host_star_name: 56.2% missing
  discovery_year: 56.2% missing
  num_planets: 56.2% missing
  discovery_method: 56.2% missing
  num_stars: 56.2% missing

🔍 CHECKING FOR ZERO-ONLY COLUMNS:
-----------------------------------
Columns with only zeros: 0

🔍 CHECKING FOR ZERO+NA ONLY COLUMNS:
----------------------------------------
Columns with only zeros and NAs: 0

In [27]:
# PERFORM TYPE CONVERSIONS AND DROP PROBLEMATIC COLUMNS
print(f"\n🧹 CLEANING AND TYPE CONVERSION")
print("=" * 35)

# Create a copy for cleaning
master_df_clean = master_df.copy()

# 1. DROP COLUMNS WITH 100% MISSING VALUES
columns_to_drop_100_missing = ['k2_name', 'planet_mass_earth', 'planet_mass_jupiter', 'stellar_age']
print(f"Dropping {len(columns_to_drop_100_missing)} columns with 100% missing values:")
for col in columns_to_drop_100_missing:
    if col in master_df_clean.columns:
        print(f"  ✗ {col}")
        master_df_clean = master_df_clean.drop(columns=[col])

# 2. IDENTIFY COLUMNS TO DROP (>90% missing)
high_missing_threshold = 90
columns_to_drop_high_missing = missing_analysis[missing_analysis['Missing_Percentage'] > high_missing_threshold]['Column'].tolist()
columns_to_drop_high_missing = [col for col in columns_to_drop_high_missing if col not in columns_to_drop_100_missing]

if columns_to_drop_high_missing:
    print(f"\nDropping {len(columns_to_drop_high_missing)} additional columns with >{high_missing_threshold}% missing:")
    for col in columns_to_drop_high_missing:
        if col in master_df_clean.columns:
            print(f"  ✗ {col}")
            master_df_clean = master_df_clean.drop(columns=[col])

# 3. PROPER TYPE CONVERSIONS
print(f"\n🔄 PERFORMING TYPE CONVERSIONS:")
print("-" * 32)

# Define categorical columns
categorical_columns = ['data_source', 'mission', 'disposition', 'disposition_reference']

# Define string/object columns that should remain as strings
string_columns = ['combined_id', 'planet_name', 'host_star_name', 'kepid', 'koi_name', 
                 'discovery_method', 'discovery_facility']

# Convert categorical columns
for col in categorical_columns:
    if col in master_df_clean.columns:
        master_df_clean[col] = master_df_clean[col].astype('category')
        print(f"  ✓ {col} → category")

# Convert string columns to proper string type
for col in string_columns:
    if col in master_df_clean.columns and master_df_clean[col].dtype != 'string':
        master_df_clean[col] = master_df_clean[col].astype('string')
        print(f"  ✓ {col} → string")

# Ensure numeric columns are proper numeric types
numeric_columns = [
    'orbital_period', 'orbital_period_err1', 'orbital_period_err2', 'semi_major_axis', 
    'eccentricity', 'inclination', 'planet_radius_earth', 'planet_radius_earth_err1', 
    'planet_radius_earth_err2', 'equilibrium_temperature', 'insolation_flux',
    'transit_epoch', 'transit_duration', 'transit_depth', 'impact_parameter',
    'stellar_effective_temp', 'stellar_radius', 'stellar_mass', 'stellar_metallicity',
    'stellar_surface_gravity', 'kepler_magnitude', 'v_magnitude', 'j_magnitude',
    'h_magnitude', 'k_magnitude', 'gaia_magnitude', 'ra', 'dec', 'distance',
    'max_single_event_stat', 'max_multiple_event_stat', 'signal_to_noise', 'num_transits'
]

for col in numeric_columns:
    if col in master_df_clean.columns:
        try:
            master_df_clean[col] = pd.to_numeric(master_df_clean[col], errors='coerce')
            print(f"  ✓ {col} → numeric")
        except:
            print(f"  ⚠ {col} → conversion failed")

# Convert integer columns
integer_columns = ['row_id', 'num_stars', 'num_planets', 'discovery_year']
for col in integer_columns:
    if col in master_df_clean.columns:
        # Convert to nullable integer
        master_df_clean[col] = master_df_clean[col].astype('Int64')
        print(f"  ✓ {col} → Int64")

print(f"\n📊 FINAL CLEANING SUMMARY:")
print("-" * 30)
print(f"Original shape: {master_df.shape}")
print(f"Cleaned shape: {master_df_clean.shape}")
print(f"Columns dropped: {master_df.shape[1] - master_df_clean.shape[1]}")
print(f"Rows retained: {len(master_df_clean):,}")

# Check final data types
print(f"\nFinal data type distribution:")
final_dtypes = master_df_clean.dtypes.value_counts()
print(final_dtypes)

# Update the master dataframe
master_df = master_df_clean.copy()

print(f"\n✅ TYPE CONVERSION AND CLEANING COMPLETE!")
print(f"Final dataset: {master_df.shape[0]:,} rows × {master_df.shape[1]} columns")


🧹 CLEANING AND TYPE CONVERSION
Dropping 4 columns with 100% missing values:
  ✗ k2_name
  ✗ planet_mass_earth
  ✗ planet_mass_jupiter
  ✗ stellar_age

🔄 PERFORMING TYPE CONVERSIONS:
--------------------------------
  ✓ data_source → category
  ✓ mission → category
  ✓ disposition → category
  ✓ disposition_reference → category
  ✓ combined_id → string
  ✓ planet_name → string
  ✓ host_star_name → string
  ✓ kepid → string
  ✓ koi_name → string
  ✓ discovery_method → string
  ✓ discovery_facility → string
  ✓ orbital_period → numeric
  ✓ orbital_period_err1 → numeric
  ✓ orbital_period_err2 → numeric
  ✓ semi_major_axis → numeric
  ✓ eccentricity → numeric
  ✓ inclination → numeric
  ✓ planet_radius_earth → numeric
  ✓ planet_radius_earth_err1 → numeric
  ✓ planet_radius_earth_err2 → numeric
  ✓ equilibrium_temperature → numeric
  ✓ insolation_flux → numeric
  ✓ transit_epoch → numeric
  ✓ transit_duration → numeric
  ✓ transit_depth → numeric
  ✓ impact_parameter → numeric
  ✓ stellar

In [28]:
# FINAL VALIDATION AND SUMMARY
print(f"\n🎯 FINAL DATASET VALIDATION")
print("=" * 30)

# Show remaining columns and their completeness
remaining_missing = pd.DataFrame({
    'Column': master_df.columns,
    'Data_Type': master_df.dtypes,
    'Non_Null_Count': master_df.count(),
    'Missing_Count': master_df.isnull().sum(),
    'Missing_Percentage': (master_df.isnull().sum() / len(master_df)) * 100,
    'Completeness': ((master_df.count() / len(master_df)) * 100).round(1)
})

# Sort by completeness (descending)
remaining_missing = remaining_missing.sort_values('Completeness', ascending=False)

print(f"📊 COLUMN COMPLETENESS SUMMARY:")
print("-" * 35)
print(f"Total columns: {len(master_df.columns)}")

# Show columns by completeness categories
complete_cols = remaining_missing[remaining_missing['Completeness'] == 100.0]
high_complete = remaining_missing[(remaining_missing['Completeness'] >= 80) & (remaining_missing['Completeness'] < 100)]
medium_complete = remaining_missing[(remaining_missing['Completeness'] >= 50) & (remaining_missing['Completeness'] < 80)]
low_complete = remaining_missing[remaining_missing['Completeness'] < 50]

print(f"✅ 100% complete: {len(complete_cols)} columns")
print(f"🟢 80-99% complete: {len(high_complete)} columns") 
print(f"🟡 50-79% complete: {len(medium_complete)} columns")
print(f"🔴 <50% complete: {len(low_complete)} columns")

# Show the most important/complete columns
print(f"\n📋 KEY COLUMNS (100% complete):")
for _, row in complete_cols.head(10).iterrows():
    print(f"  ✓ {row['Column']} ({row['Data_Type']})")

# Memory usage
memory_usage = master_df.memory_usage(deep=True).sum() / 1024**2  # Convert to MB
print(f"\n💾 MEMORY USAGE: {memory_usage:.2f} MB")

# Quick statistics for key numeric columns
key_numeric_cols = ['orbital_period', 'planet_radius_earth', 'equilibrium_temperature', 
                   'stellar_effective_temp', 'kepler_magnitude']
available_key_cols = [col for col in key_numeric_cols if col in master_df.columns]

if available_key_cols:
    print(f"\n📈 KEY STATISTICS:")
    key_stats = master_df[available_key_cols].describe().round(3)
    print(key_stats)

print(f"\n🎉 DATASET READY FOR ANALYSIS!")
print(f"   - {len(master_df):,} exoplanet candidates and confirmed planets")
print(f"   - {len(master_df.columns)} high-quality features")
print(f"   - Proper data types assigned")
print(f"   - Low-quality columns removed")
print(f"   - Ready for machine learning and scientific analysis")


🎯 FINAL DATASET VALIDATION
📊 COLUMN COMPLETENESS SUMMARY:
-----------------------------------
Total columns: 51
✅ 100% complete: 7 columns
🟢 80-99% complete: 10 columns
🟡 50-79% complete: 22 columns
🔴 <50% complete: 12 columns

📋 KEY COLUMNS (100% complete):
  ✓ data_source (category)
  ✓ disposition (category)
  ✓ row_id (Int64)
  ✓ mission (category)
  ✓ dec (float64)
  ✓ combined_id (string)
  ✓ ra (float64)

💾 MEMORY USAGE: 6.56 MB

📈 KEY STATISTICS:
       orbital_period  planet_radius_earth  equilibrium_temperature  \
count        8372.000             7694.000                 5461.000   
mean           67.627               27.988                  837.144   
std          1696.532             1302.941                  467.495   
min             0.176                0.220                   25.000   
25%             4.175                1.500                  517.000   
50%             9.762                2.240                  763.000   
75%            23.239                3.398 

In [29]:
# Define low importance features to drop (based on feature importance analysis)
low_importance_features = [
    'stellar_surface_gravity',    # 0.0094 (0.9%)
    'j_magnitude',               # 0.0092 (0.9%)
    'kepler_magnitude',          # 0.0092 (0.9%)
    'h_magnitude',               # 0.0085 (0.9%)
    'planet_radius_jupiter',     # 0.0085 (0.8%)
    'discovery_year',            # 0.0062 (0.6%)
    'eccentricity',              # 0.0008 (0.1%)
    'num_stars'                  # 0.0004 (0.0%)
]

In [30]:
# RANDOM FOREST MACHINE LEARNING MODEL
print("RANDOM FOREST TRAINING AND EVALUATION")
print("=" * 45)

# Import required libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import (accuracy_score, roc_auc_score, classification_report, 
                           confusion_matrix, precision_recall_fscore_support)
import numpy as np

# Prepare features and target
print("Preparing data for machine learning...")

# Select numeric features for modeling
numeric_features = master_df.select_dtypes(include=[np.number]).columns.tolist()
# Remove ID columns and target-related columns
exclude_columns = ['row_id']
feature_columns = [col for col in numeric_features if col not in exclude_columns]

# Prepare feature matrix
X = master_df[feature_columns].copy()
print(f"Selected {len(feature_columns)} features for modeling")

# Drop low importance features identified from feature importance analysis
X = X.drop(columns=low_importance_features, errors='ignore')
print(f"After dropping {len(low_importance_features)} low importance features: {X.shape[1]} features remaining")

# Prepare target variable (encode disposition as binary)
# CONFIRMED = 1, CANDIDATE = 0
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(master_df['disposition'])
target_classes = label_encoder.classes_

print(f"Target encoding: {target_classes[0]} = 0, {target_classes[1]} = 1")
print(f"Class distribution:")
unique, counts = np.unique(y, return_counts=True)
for i, (cls, count) in enumerate(zip(unique, counts)):
    percentage = (count / len(y)) * 100
    print(f"  {target_classes[i]}: {count:,} ({percentage:.1f}%)")

print(f"\nOriginal missing values: {X.isnull().sum().sum()}")

# Handle missing values using imputation
imputer = SimpleImputer(strategy='median')
X_imputed = pd.DataFrame(
    imputer.fit_transform(X), 
    columns=X.columns, 
    index=X.index
)

print(f"Remaining missing values: {X_imputed.isnull().sum().sum()}")

RANDOM FOREST TRAINING AND EVALUATION
Preparing data for machine learning...
Selected 38 features for modeling
After dropping 8 low importance features: 30 features remaining
Target encoding: CANDIDATE = 0, CONFIRMED = 1
Class distribution:
  CANDIDATE: 3,353 (39.9%)
  CONFIRMED: 5,061 (60.1%)

Original missing values: 77412
Remaining missing values: 0


In [31]:
# Train-test split and model training
test_size = 0.2
random_state = 42

X_train, X_test, y_train, y_test = train_test_split(
    X_imputed, y, 
    test_size=test_size, 
    random_state=random_state, 
    stratify=y
)

print(f"\nTraining set: {X_train.shape[0]:,} samples")
print(f"Test set: {X_test.shape[0]:,} samples")
print(f"Features: {X_train.shape[1]}")

# Check class distribution in splits
print(f"\nClass distribution in training set:")
train_classes, train_counts = np.unique(y_train, return_counts=True)
for cls, count in zip(train_classes, train_counts):
    class_name = target_classes[cls]
    percentage = (count / len(y_train)) * 100
    print(f"  {class_name}: {count:,} ({percentage:.1f}%)")

print(f"\nClass distribution in test set:")
test_classes, test_counts = np.unique(y_test, return_counts=True)
for cls, count in zip(test_classes, test_counts):
    class_name = target_classes[cls]
    percentage = (count / len(y_test)) * 100
    print(f"  {class_name}: {count:,} ({percentage:.1f}%)")

# Create and train Random Forest model
rf_model = RandomForestClassifier(
    n_estimators=100,
    random_state=random_state,
    n_jobs=-1  # Use all available cores
)

print(f"\nTraining Random Forest model...")
rf_model.fit(X_train, y_train)

# Make predictions
y_train_pred = rf_model.predict(X_train)
y_test_pred = rf_model.predict(X_test)
y_test_proba = rf_model.predict_proba(X_test)[:, 1]  # Probability for CONFIRMED class

# Calculate basic metrics
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_auc = roc_auc_score(y_test, y_test_proba)

print("Random Forest model trained successfully")
print(f"Training accuracy: {train_accuracy:.4f}")
print(f"Test accuracy: {test_accuracy:.4f}")
print(f"Test AUC-ROC: {test_auc:.4f}")

# Cross-validation
cv_scores = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='accuracy')
print(f"Cross-validation accuracy: {cv_scores.mean():.4f} (±{cv_scores.std()*2:.4f})")


Training set: 6,731 samples
Test set: 1,683 samples
Features: 30

Class distribution in training set:
  CANDIDATE: 2,682 (39.8%)
  CONFIRMED: 4,049 (60.2%)

Class distribution in test set:
  CANDIDATE: 671 (39.9%)
  CONFIRMED: 1,012 (60.1%)

Training Random Forest model...
Random Forest model trained successfully
Training accuracy: 1.0000
Test accuracy: 0.9299
Test AUC-ROC: 0.9739
Cross-validation accuracy: 0.9326 (±0.0147)


In [32]:
# DETAILED PERFORMANCE METRICS
print("\nDETAILED PERFORMANCE ANALYSIS")
print("=" * 40)

# Classification Report with Precision and Recall
print("\nClassification Report:")
class_names = ['CANDIDATE', 'CONFIRMED']
report = classification_report(y_test, y_test_pred, target_names=class_names, digits=4)
print(report)

# Precision, Recall, F1-score by class
precision, recall, f1, support = precision_recall_fscore_support(y_test, y_test_pred, average=None)
print("\nDetailed Metrics by Class:")
for i, class_name in enumerate(class_names):
    print(f"{class_name}:")
    print(f"  Precision: {precision[i]:.4f}")
    print(f"  Recall: {recall[i]:.4f}")
    print(f"  F1-Score: {f1[i]:.4f}")
    print(f"  Support: {support[i]}")

# Macro and Micro averages
macro_precision, macro_recall, macro_f1, _ = precision_recall_fscore_support(y_test, y_test_pred, average='macro')
micro_precision, micro_recall, micro_f1, _ = precision_recall_fscore_support(y_test, y_test_pred, average='micro')

print(f"\nMacro Average:")
print(f"  Precision: {macro_precision:.4f}")
print(f"  Recall: {macro_recall:.4f}")
print(f"  F1-Score: {macro_f1:.4f}")

print(f"\nMicro Average:")
print(f"  Precision: {micro_precision:.4f}")
print(f"  Recall: {micro_recall:.4f}")
print(f"  F1-Score: {micro_f1:.4f}")


DETAILED PERFORMANCE ANALYSIS

Classification Report:
              precision    recall  f1-score   support

   CANDIDATE     0.9382    0.8823    0.9094       671
   CONFIRMED     0.9249    0.9615    0.9428      1012

    accuracy                         0.9299      1683
   macro avg     0.9315    0.9219    0.9261      1683
weighted avg     0.9302    0.9299    0.9295      1683


Detailed Metrics by Class:
CANDIDATE:
  Precision: 0.9382
  Recall: 0.8823
  F1-Score: 0.9094
  Support: 671
CONFIRMED:
  Precision: 0.9249
  Recall: 0.9615
  F1-Score: 0.9428
  Support: 1012

Macro Average:
  Precision: 0.9315
  Recall: 0.9219
  F1-Score: 0.9261

Micro Average:
  Precision: 0.9299
  Recall: 0.9299
  F1-Score: 0.9299


In [33]:
# CONFUSION MATRIX ANALYSIS
print("\nCONFUSION MATRIX")
print("=" * 20)
cm = confusion_matrix(y_test, y_test_pred)
print("\nConfusion Matrix:")
print(f"                Predicted")
print(f"                CANDIDATE  CONFIRMED")
print(f"Actual CANDIDATE     {cm[0,0]:4d}      {cm[0,1]:4d}")
print(f"       CONFIRMED     {cm[1,0]:4d}      {cm[1,1]:4d}")

# Calculate confusion matrix percentages
cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
print("\nConfusion Matrix (Percentages):")
print(f"                Predicted")
print(f"                CANDIDATE  CONFIRMED")
print(f"Actual CANDIDATE    {cm_percent[0,0]:5.1f}%     {cm_percent[0,1]:5.1f}%")
print(f"       CONFIRMED    {cm_percent[1,0]:5.1f}%     {cm_percent[1,1]:5.1f}%")

# True/False Positives and Negatives
tn, fp, fn, tp = cm.ravel()
print(f"\nConfusion Matrix Components:")
print(f"True Negatives (TN): {tn}")
print(f"False Positives (FP): {fp}")
print(f"False Negatives (FN): {fn}")
print(f"True Positives (TP): {tp}")

# Additional metrics from confusion matrix
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)  # Same as recall
print(f"\nAdditional Metrics:")
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"False Positive Rate: {fp/(fp+tn):.4f}")
print(f"False Negative Rate: {fn/(fn+tp):.4f}")


CONFUSION MATRIX

Confusion Matrix:
                Predicted
                CANDIDATE  CONFIRMED
Actual CANDIDATE      592        79
       CONFIRMED       39       973

Confusion Matrix (Percentages):
                Predicted
                CANDIDATE  CONFIRMED
Actual CANDIDATE     88.2%      11.8%
       CONFIRMED      3.9%      96.1%

Confusion Matrix Components:
True Negatives (TN): 592
False Positives (FP): 79
False Negatives (FN): 39
True Positives (TP): 973

Additional Metrics:
Sensitivity (Recall): 0.9615
Specificity: 0.8823
False Positive Rate: 0.1177
False Negative Rate: 0.0385


In [34]:
# FEATURE IMPORTANCE ANALYSIS
print("\nFEATURE IMPORTANCE ANALYSIS")
print("=" * 35)

# Get feature importances
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 20 Most Important Features:")
print("-" * 45)
for i, (_, row) in enumerate(feature_importance.head(20).iterrows(), 1):
    print(f"{i:2d}. {row['feature']:<30} {row['importance']:.4f}")

# Feature importance statistics
print(f"\nFeature Importance Statistics:")
print(f"Total features: {len(feature_importance)}")
print(f"Mean importance: {feature_importance['importance'].mean():.4f}")
print(f"Std importance: {feature_importance['importance'].std():.4f}")
print(f"Max importance: {feature_importance['importance'].max():.4f}")
print(f"Min importance: {feature_importance['importance'].min():.4f}")

# Features with importance > mean
important_features = feature_importance[feature_importance['importance'] > feature_importance['importance'].mean()]
print(f"\nFeatures above average importance: {len(important_features)}")

# Features contributing to 80% of total importance
cumulative_importance = feature_importance['importance'].cumsum() / feature_importance['importance'].sum()
top_80_count = (cumulative_importance <= 0.8).sum() + 1
print(f"Features contributing to 80% of importance: {top_80_count}")

print("\nTop 10 Most Important Features for Prediction:")
print("-" * 50)
for i, (_, row) in enumerate(feature_importance.iterrows(), 1):
    importance_pct = (row['importance'] / feature_importance['importance'].sum()) * 100
    print(f"{i:2d}. {row['feature']:<30} {row['importance']:.4f} ({importance_pct:.1f}%)")

print("\nModel training and evaluation completed successfully")
print("Final model ready for predictions and further analysis")


FEATURE IMPORTANCE ANALYSIS

Top 20 Most Important Features:
---------------------------------------------
 1. num_planets                    0.2601
 2. max_multiple_event_stat        0.1157
 3. signal_to_noise                0.1145
 4. orbital_period                 0.0308
 5. disposition_score              0.0301
 6. planet_radius_earth            0.0299
 7. planet_radius_earth_err1       0.0290
 8. dec                            0.0273
 9. orbital_period_err1            0.0248
10. semi_major_axis                0.0236
11. planet_radius_earth_err2       0.0235
12. orbital_period_err2            0.0226
13. transit_duration               0.0195
14. max_single_event_stat          0.0194
15. ra                             0.0192
16. stellar_metallicity            0.0182
17. transit_depth                  0.0160
18. stellar_radius                 0.0156
19. k_magnitude                    0.0151
20. num_transits                   0.0149

Feature Importance Statistics:
Total features: 30
M

In [37]:

from sklearn.model_selection import GridSearchCV
# HYPERPARAMETER TUNING
print(f"\n⚙️ HYPERPARAMETER TUNING:")
print("-" * 28)

# Define parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

print(f"Parameter grid defined with {np.prod([len(v) for v in param_grid.values()])} combinations")

# Perform grid search with cross-validation
rf_grid = RandomForestClassifier(random_state=random_state, n_jobs=-1)
grid_search = GridSearchCV(
    rf_grid, 
    param_grid, 
    cv=3,  # Use 3-fold CV for speed
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

print("🔍 Starting grid search...")
grid_search.fit(X_train, y_train)

# Get best model
best_rf = grid_search.best_estimator_
print(f"\n✅ Grid search completed!")
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation AUC: {grid_search.best_score_:.4f}")

# EVALUATE BEST MODEL
print(f"\n📊 EVALUATING BEST MODEL:")
print("-" * 27)

# Make predictions with best model
y_train_pred_best = best_rf.predict(X_train)
y_test_pred_best = best_rf.predict(X_test)
y_test_proba_best = best_rf.predict_proba(X_test)[:, 1]

# Calculate metrics
train_accuracy_best = accuracy_score(y_train, y_train_pred_best)
test_accuracy_best = accuracy_score(y_test, y_test_pred_best)
test_auc_best = roc_auc_score(y_test, y_test_proba_best)

print(f"Best model performance:")
print(f"  Training accuracy: {train_accuracy_best:.4f}")
print(f"  Test accuracy: {test_accuracy_best:.4f}")
print(f"  Test AUC-ROC: {test_auc_best:.4f}")

# Detailed classification report
print(f"\n📋 DETAILED CLASSIFICATION REPORT:")
print("-" * 37)
class_report = classification_report(
    y_test, y_test_pred_best, 
    target_names=target_classes,
    digits=4
)
print(class_report)

# Confusion Matrix
print(f"\n🔍 CONFUSION MATRIX:")
print("-" * 19)
cm = confusion_matrix(y_test, y_test_pred_best)
print("Actual\\Predicted  CANDIDATE  CONFIRMED")
print(f"CANDIDATE         {cm[0,0]:9d}  {cm[0,1]:9d}")
print(f"CONFIRMED         {cm[1,0]:9d}  {cm[1,1]:9d}")

# Calculate additional metrics
tn, fp, fn, tp = cm.ravel()
precision_confirmed = tp / (tp + fp) if (tp + fp) > 0 else 0
recall_confirmed = tp / (tp + fn) if (tp + fn) > 0 else 0
precision_candidate = tn / (tn + fn) if (tn + fn) > 0 else 0
recall_candidate = tn / (tn + fp) if (tn + fp) > 0 else 0

print(f"\n📈 ADDITIONAL METRICS:")
print(f"  Precision (CONFIRMED): {precision_confirmed:.4f}")
print(f"  Recall (CONFIRMED):    {recall_confirmed:.4f}")
print(f"  Precision (CANDIDATE): {precision_candidate:.4f}")
print(f"  Recall (CANDIDATE):    {recall_candidate:.4f}")


⚙️ HYPERPARAMETER TUNING:
----------------------------
Parameter grid defined with 216 combinations
🔍 Starting grid search...
Fitting 3 folds for each of 216 candidates, totalling 648 fits

✅ Grid search completed!
Best parameters: {'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Best cross-validation AUC: 0.9794

📊 EVALUATING BEST MODEL:
---------------------------
Best model performance:
  Training accuracy: 1.0000
  Test accuracy: 0.9293
  Test AUC-ROC: 0.9751

📋 DETAILED CLASSIFICATION REPORT:
-------------------------------------
              precision    recall  f1-score   support

   CANDIDATE     0.9340    0.8852    0.9090       671
   CONFIRMED     0.9265    0.9585    0.9422      1012

    accuracy                         0.9293      1683
   macro avg     0.9302    0.9219    0.9256      1683
weighted avg     0.9294    0.9293    0.9289      1683


🔍 CONFUSION MATRIX:
-------------------
Actual\Predicted  CANDIDATE  

In [39]:
# SAVE TRAINED MODEL AS JOBLIB FILE
print("SAVING RANDOM FOREST MODEL")
print("=" * 30)

import joblib
import os

# Create models directory if it doesn't exist
models_dir = "models"
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
    print(f"Created directory: {models_dir}")

# Save the trained Random Forest model
model_filename = "merged_dataset_rf_model.joblib"
model_path = os.path.join(models_dir, model_filename)

# Save the best Random Forest model
joblib.dump(best_rf, model_path)

print(f"✅ Model saved successfully as: {model_path}")
print(f"📊 Model type: {type(best_rf).__name__}")
print(f"🎯 Model parameters: {best_rf.get_params()}")

# Also save additional components that might be needed for predictions
components_filename = "merged_dataset_rf_components.joblib"
components_path = os.path.join(models_dir, components_filename)

# Save imputer, label encoder, and feature columns for future predictions
model_components = {
    'model': best_rf,
    'imputer': imputer,
    'label_encoder': label_encoder,
    'feature_columns': X.columns.tolist(),
    'target_classes': target_classes,
    'low_importance_features': low_importance_features
}

joblib.dump(model_components, components_path)

print(f"✅ Model components saved as: {components_path}")
print(f"📋 Components included: {list(model_components.keys())}")

# Display file sizes
model_size = os.path.getsize(model_path) / 1024  # KB
components_size = os.path.getsize(components_path) / 1024  # KB

print(f"\n📁 File sizes:")
print(f"   Model only: {model_size:.1f} KB")
print(f"   Full components: {components_size:.1f} KB")

print(f"\n💡 To load the model later:")
print(f"   model = joblib.load('{model_path}')")
print(f"   components = joblib.load('{components_path}')")

SAVING RANDOM FOREST MODEL
✅ Model saved successfully as: models\merged_dataset_rf_model.joblib
📊 Model type: RandomForestClassifier
🎯 Model parameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 30, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 300, 'n_jobs': -1, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}
✅ Model components saved as: models\merged_dataset_rf_components.joblib
📋 Components included: ['model', 'imputer', 'label_encoder', 'feature_columns', 'target_classes', 'low_importance_features']

📁 File sizes:
   Model only: 18505.3 KB
   Full components: 18507.8 KB

💡 To load the model later:
   model = joblib.load('models\merged_dataset_rf_model.joblib')
   components = joblib.load('models\merged_dataset_rf_components.joblib')
✅ 